In [106]:
library(ncdf4)

# Read the netCDF file
nc_data <- nc_open("/cluster/home/haroldh/spGDMM/1_data/4_interim/SINMOD_features.nc")

var_names <- names(nc_data$var)

var_names

[1] "temperature" "salinity"

In [107]:
# Process each variable and reshape it
midnor_env_predictors <- lapply(var_names, function(var) {
  # Extract data for the variable
  var_data <- ncvar_get(nc_data, var)  # Shape: x, y, stat
  
  # Reshape to (grid cells, stat)
  dim(var_data) <- c(prod(dim(var_data)[1:2]), dim(var_data)[3])
  
  # Get the 'vals' for the 'stat' dimension (if it exists)
  stat_vals <- nc_data$dim$stat$vals  # Adjust if the dimension name isn't 'stat'
  
  # Check if stat_vals exists; otherwise, use generic names
  if (is.null(stat_vals)) {
    stat_vals <- paste0("stat", seq_len(dim(var_data)[2]))
  }
  
  # Assign column names using the variable name and stat_vals
  colnames(var_data) <- paste0(var, "_", stat_vals)
  
  # Convert to a data frame
  return(as.data.frame(var_data))
})

# Combine all variables into a single data frame
midnor_env_predictors <- do.call(cbind, midnor_env_predictors)

# Print column names to verify
print(colnames(midnor_env_predictors))

[1] "temperature_mean"            "temperature_10th_percentile"
[3] "temperature_90th_percentile" "salinity_mean"              
[5] "salinity_10th_percentile"    "salinity_90th_percentile"   


In [108]:
library(splines2)

# Define degree and number of knots
degree <- 3
df <- 4  # Degrees of freedom (including intercept)

# Efficiently compute I-spline bases for all features
i_spline_matrix <- do.call(
  cbind,
  lapply(colnames(midnor_env_predictors), function(col) {
    # Apply I-spline to the feature
    spline_matrix <- iSpline(
      midnor_env_predictors[[col]],
      degree = degree - 1,
      df = df,
      intercept = TRUE,
      skipna = TRUE
    )
    
    # Generate descriptive column names for this feature
    colnames(spline_matrix) <- paste0(col, "_I", seq_len(df))  # Remaining bases
    
    return(spline_matrix)
  })
)

# Print dimensions and column names for verification
print(dim(i_spline_matrix))  # Dimensions: (num rows, num features * df)
print(colnames(i_spline_matrix))  # Column names: feature_Intercept, feature_I1, ...

write.csv(i_spline_matrix, "/cluster/home/haroldh/spGDMM/1_data/2_processed/prediction/X_GDM_predictors_bs.csv", row.names = FALSE)

[1] 525746     24
 [1] "temperature_mean_I1"            "temperature_mean_I2"           
 [3] "temperature_mean_I3"            "temperature_mean_I4"           
 [5] "temperature_10th_percentile_I1" "temperature_10th_percentile_I2"
 [7] "temperature_10th_percentile_I3" "temperature_10th_percentile_I4"
 [9] "temperature_90th_percentile_I1" "temperature_90th_percentile_I2"
[11] "temperature_90th_percentile_I3" "temperature_90th_percentile_I4"
[13] "salinity_mean_I1"               "salinity_mean_I2"              
[15] "salinity_mean_I3"               "salinity_mean_I4"              
[17] "salinity_10th_percentile_I1"    "salinity_10th_percentile_I2"   
[19] "salinity_10th_percentile_I3"    "salinity_10th_percentile_I4"   
[21] "salinity_90th_percentile_I1"    "salinity_90th_percentile_I2"   
[23] "salinity_90th_percentile_I3"    "salinity_90th_percentile_I4"   


In [100]:
formula_use = as.formula(paste("~ 0 +",paste(
  paste("iSpline(`",colnames(midnor_env_predictors),"`,degree=",degree - 1 ,",df = ",df, 
        " ,intercept = TRUE)",sep = ""),collapse = "+")))

I_spline_bases = model.matrix(formula_use, data = midnor_env_predictors)

In [101]:
I_spline_bases

,"iSpline(temperature_mean, degree = 2, df = 5, intercept = TRUE)1","iSpline(temperature_mean, degree = 2, df = 5, intercept = TRUE)2","iSpline(temperature_mean, degree = 2, df = 5, intercept = TRUE)3","iSpline(temperature_mean, degree = 2, df = 5, intercept = TRUE)4","iSpline(temperature_mean, degree = 2, df = 5, intercept = TRUE)5","iSpline(temperature_10th_percentile, degree = 2, df = 5, intercept = TRUE)1","iSpline(temperature_10th_percentile, degree = 2, df = 5, intercept = TRUE)2","iSpline(temperature_10th_percentile, degree = 2, df = 5, intercept = TRUE)3","iSpline(temperature_10th_percentile, degree = 2, df = 5, intercept = TRUE)4","iSpline(temperature_10th_percentile, degree = 2, df = 5, intercept = TRUE)5",⋯,"iSpline(salinity_10th_percentile, degree = 2, df = 5, intercept = TRUE)1","iSpline(salinity_10th_percentile, degree = 2, df = 5, intercept = TRUE)2","iSpline(salinity_10th_percentile, degree = 2, df = 5, intercept = TRUE)3","iSpline(salinity_10th_percentile, degree = 2, df = 5, intercept = TRUE)4","iSpline(salinity_10th_percentile, degree = 2, df = 5, intercept = TRUE)5","iSpline(salinity_90th_percentile, degree = 2, df = 5, intercept = TRUE)1","iSpline(salinity_90th_percentile, degree = 2, df = 5, intercept = TRUE)2","iSpline(salinity_90th_percentile, degree = 2, df = 5, intercept = TRUE)3","iSpline(salinity_90th_percentile, degree = 2, df = 5, intercept = TRUE)4","iSpline(salinity_90th_percentile, degree = 2, df = 5, intercept = TRUE)5"
39,0.9656115,0.6880278,0.20492731,0.000000000,0.000000000,1.0000000,1.0000000,0.779575850,2.311526e-01,2.874345e-03,⋯,0.00000000,0.000000000,0.000000e+00,0.0000000000,0.000000000,0.9532780,0.6268075,0.1700104,0.000000e+00,0.000000000
40,0.9829495,0.7725188,0.27311903,0.000000000,0.000000000,1.0000000,0.8900487,0.227882763,2.848352e-03,0.000000e+00,⋯,0.91282246,0.456830661,7.133584e-02,0.0000000000,0.000000000,0.9988171,0.9099205,0.4641375,0.000000e+00,0.000000000
49,1.0000000,0.9931353,0.74821239,0.004530651,0.000000000,1.0000000,1.0000000,0.689950859,1.403906e-01,5.869577e-05,⋯,0.99186878,0.748756522,2.109549e-01,0.0000000000,0.000000000,0.9999522,0.9570646,0.5809348,0.000000e+00,0.000000000
50,1.0000000,1.0000000,0.99197388,0.703120150,0.113811810,1.0000000,1.0000000,0.971617114,7.157915e-01,1.821181e-01,⋯,0.98919275,0.726830081,1.955022e-01,0.0000000000,0.000000000,0.9999953,0.9670611,0.6169602,0.000000e+00,0.000000000
58,1.0000000,1.0000000,0.96031516,0.349230677,0.001814837,1.0000000,1.0000000,0.919085641,5.041505e-01,5.741856e-02,⋯,1.00000000,0.956133541,4.999063e-01,0.0013332413,0.000000000,1.0000000,0.9987158,0.8813759,1.053480e-01,0.000000000
59,1.0000000,1.0000000,0.96697500,0.397412111,0.005279819,1.0000000,1.0000000,0.960784171,6.601275e-01,1.386210e-01,⋯,0.99999624,0.913191867,3.946979e-01,0.0000000000,0.000000000,1.0000000,0.9922037,0.7737717,1.467059e-02,0.000000000
60,1.0000000,1.0000000,1.00000000,1.000000000,1.000000000,1.0000000,1.0000000,1.000000000,1.000000e+00,1.000000e+00,⋯,0.99850225,0.835458977,2.873321e-01,0.0000000000,0.000000000,1.0000000,0.9729573,0.6422071,0.000000e+00,0.000000000
66,1.0000000,1.0000000,0.99073960,0.678962152,0.096983397,1.0000000,0.9848981,0.434208928,2.887396e-02,0.000000e+00,⋯,1.00000000,1.000000000,9.278830e-01,0.3506301206,0.001145690,1.0000000,1.0000000,0.9999799,9.919084e-01,0.755402964
67,1.0000000,1.0000000,0.99717733,0.836950520,0.257514485,1.0000000,0.9988595,0.563911427,6.808823e-02,0.000000e+00,⋯,1.00000000,0.999410659,8.245836e-01,0.1373758990,0.000000000,1.0000000,1.0000000,0.9947437,7.440675e-01,0.079126746
68,1.0000000,1.0000000,0.97038852,0.425575554,0.008442948,1.0000000,1.0000000,0.730258497,1.759985e-01,5.607251e-04,⋯,1.00000000,0.977353623,5.876137e-01,0.0099922080,0.000000000,1.0000000,0.9998724,0.9373781,2.388721e-01,0.000000000


In [102]:
i_spline_matrix

temperature_mean_I1,temperature_mean_I2,temperature_mean_I3,temperature_mean_I4,temperature_mean_I5,temperature_10th_percentile_I1,temperature_10th_percentile_I2,temperature_10th_percentile_I3,temperature_10th_percentile_I4,temperature_10th_percentile_I5,⋯,salinity_10th_percentile_I1,salinity_10th_percentile_I2,salinity_10th_percentile_I3,salinity_10th_percentile_I4,salinity_10th_percentile_I5,salinity_90th_percentile_I1,salinity_90th_percentile_I2,salinity_90th_percentile_I3,salinity_90th_percentile_I4,salinity_90th_percentile_I5
NA,NA,NA,NA,NA,NA,NA,NA,NA,NA,⋯,NA,NA,NA,NA,NA,NA,NA,NA,NA,NA
NA,NA,NA,NA,NA,NA,NA,NA,NA,NA,⋯,NA,NA,NA,NA,NA,NA,NA,NA,NA,NA
NA,NA,NA,NA,NA,NA,NA,NA,NA,NA,⋯,NA,NA,NA,NA,NA,NA,NA,NA,NA,NA
NA,NA,NA,NA,NA,NA,NA,NA,NA,NA,⋯,NA,NA,NA,NA,NA,NA,NA,NA,NA,NA
NA,NA,NA,NA,NA,NA,NA,NA,NA,NA,⋯,NA,NA,NA,NA,NA,NA,NA,NA,NA,NA
NA,NA,NA,NA,NA,NA,NA,NA,NA,NA,⋯,NA,NA,NA,NA,NA,NA,NA,NA,NA,NA
NA,NA,NA,NA,NA,NA,NA,NA,NA,NA,⋯,NA,NA,NA,NA,NA,NA,NA,NA,NA,NA
NA,NA,NA,NA,NA,NA,NA,NA,NA,NA,⋯,NA,NA,NA,NA,NA,NA,NA,NA,NA,NA
NA,NA,NA,NA,NA,NA,NA,NA,NA,NA,⋯,NA,NA,NA,NA,NA,NA,NA,NA,NA,NA
NA,NA,NA,NA,NA,NA,NA,NA,NA,NA,⋯,NA,NA,NA,NA,NA,NA,NA,NA,NA,NA


In [95]:
library(splines2)

# Example data
feature <- c(1, 2, 3, 4, 5)

# Generate I-spline bases
spline_matrix <- iSpline(feature, degree = 2, df = 4, intercept = TRUE)

# Inspect the matrix
print(spline_matrix)


         1       2       3     4
[1,] 0.000 0.00000 0.00000 0.000
[2,] 0.875 0.28125 0.03125 0.000
[3,] 1.000 0.75000 0.25000 0.000
[4,] 1.000 0.96875 0.71875 0.125
[5,] 1.000 1.00000 1.00000 1.000


In [96]:
library(splines2)

# Example data
feature <- c(1, 2, 3, 4, 5)

# Generate I-spline bases
spline_matrix <- iSpline(feature, degree = 2, df = 4, intercept = FALSE)

# Inspect the matrix
print(spline_matrix)


             1         2         3        4
[1,] 0.0000000 0.0000000 0.0000000 0.000000
[2,] 0.5273438 0.0703125 0.0000000 0.000000
[3,] 0.9687500 0.5000000 0.0312500 0.000000
[4,] 1.0000000 0.9296875 0.4726562 0.015625
[5,] 1.0000000 1.0000000 1.0000000 1.000000


In [97]:
feature

[1] 1 2 3 4 5